In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [3]:
data = pd.read_csv("transfusion.data", names = ['last time', 'frequency of donations', 'total blood', 'first time', 'March 2007 donation'], header = 0)
data = data.sample(frac = 1).reset_index(drop = True)
dataset = data.drop(['March 2007 donation'], axis=1)
target = data['March 2007 donation']
data

,last time,frequency of donations,total blood,first time,March 2007 donation
0,4,6,1500,35,0
1,8,15,3750,77,0
2,3,5,1250,26,0
3,5,24,6000,79,0
4,23,14,3500,93,0
...,...,...,...,...,...
743,10,8,2000,39,0
744,14,4,1000,38,0
745,0,8,2000,59,0
746,2,7,1750,77,0




performance function use real target and predicted target for calculate accuracy, recall, F1_score



In [4]:
def peformance(pred_target, target):
    True_pos = True_neg = False_pos = False_neg = 0
        
    for x in range(0, len(pred_target)):                                      
        if pred_target[x] == 1 or target[x] == 1:
            True_pos += 1
        elif pred_target[x] == 0 or target[x] == 1:
             False_neg += 1    
        elif pred_target[x] == 0 or target[x] == 0:
             True_neg += 1
        else:
             False_pos += 1
        
    accuracy = ((True_pos + True_neg)/(True_pos + True_neg + False_pos + False_neg)) * 100
    try:
        precision = (True_pos / (True_pos + False_pos)) * 100
    except:
        precision = 0
    recall = (True_pos / (True_pos + False_neg)) * 100
    try:
        F1_score = ((2 * precision * recall) / (precision + recall))
    except:
        F1_score = 0
    return accuracy, recall, F1_score


calculate entropy of columns


In [5]:
def Entropy(attribute):
        names, counts = np.unique(attribute, return_counts = True)              # returns value and count of unique members
        entropy = 0
        
        for x in range(0,len(counts)):
            entropy += -(counts[x]/len(attribute)) * math.log(counts[x]/len(attribute),2)
        
        return entropy


Prepare data for decition tree

find the point that have maximum information gain and divid data to below and above point then do this for every two new parts and do this more times to changing data to discrete type.




In [6]:
def collections_divition(dataset, target, collections):
    n = len(target)
    if (collections == 0 or not dataset):
        return []
    if (collections == 1):
        return []
    
    persons = []
    for i in range(1, len(target)):
        if target[i] != target[i-1]:
            persons.append(i)
            
    if (not persons):
        return [dataset[-1]]
    
    for person in persons:
        Entropys = [(Entropy(target[:person]) * person + Entropy(target[person:]) * (n - person)) / n]
        
    best_point = persons[np.argmin(Entropys)]
    best_split = dataset[best_point]
    down_collections_number = math.floor(collections / 2)
    down_splits = collections_divition(dataset[:best_point], target[:best_point], down_collections_number)
    up_splits = collections_divition(dataset[best_point:], target[best_point:], collections - down_collections_number)
    
    return down_splits + [best_split] + up_splits

def divition(data, attribute):
    dt = data.sort_values(by=attribute, ascending=True)
    target = list(dt['March 2007 donation'])
    dataset = list(dt[attribute])
    n = len(target)
    collections = 6
    
    if  not dataset:
        return []
    
    persons = []
    for i in range(1, len(target)):
        if target[i] != target[i-1]:
            persons.append(i)
            
    for person in persons:        
        Entropys = [(Entropy(target[:person]) * person + Entropy(target[person:]) * (n - person)) / n]

    best_point = persons[np.argmin(Entropys)]
    best_split = dataset[best_point]
    down_collections_number = math.floor(collections / 2)
    down_splits = collections_divition(dataset[:best_point], target[:best_point], down_collections_number)
    up_splits = collections_divition(dataset[best_point:], target[best_point:], collections - down_collections_number)
    all_splits = sorted(list(set(down_splits + [best_split] + up_splits + [dataset[-1]])))
    dataset = data[attribute]
    
    for i in range(len(dataset)):
        for j in range(len(all_splits)):
            if dataset[i] <= all_splits[j]:
                dataset[i] = j
                break

In [7]:
features = {}
tree_data = data.copy()
tree_dataset = tree_data.drop(['March 2007 donation'], axis=1)
features = {'last time': max(tree_data['last time']) + 1,
            'frequency of donations' : max(tree_data['frequency of donations']) + 1,
            'total blood' : max(tree_data['total blood']) + 1,
            'first time' : max(tree_data['first time']) + 1}
for i in tree_dataset.columns:
    divition(tree_data, i)
    
tree_target = tree_data['March 2007 donation']

split = int(80*len(dataset)/100)
tree_train_dataset, tree_train_target = tree_dataset[:split], np.array(target[:split])
tree_test_dataset, tree_test_target = tree_dataset[split:], np.array(target[split:])

tree_test_dataset = tree_test_dataset.reset_index(drop = True)
tree_data

,last time,frequency of donations,total blood,first time,March 2007 donation
0,2,1,1,1,0
1,2,1,1,1,0
2,2,1,1,1,0
3,2,1,1,1,0
4,2,1,1,1,0
...,...,...,...,...,...
743,2,1,1,1,0
744,2,1,1,1,0
745,0,1,1,1,0
746,2,1,1,1,0



calculate information gain of each column


In [8]:
def InformationGain(dataset, target, attribute):
    n = len(target)
    total_entropy = Entropy(target)
    feature = features[attribute]
    info_gain = []
    collections = []
    for i in range(feature):
        points = dataset[attribute] == i
        new_target = target[points]
        info_gain.append((total_entropy - Entropy(new_target)) * len(points) / n)
    return sum(info_gain)


Decision Tree

Choose_best_Attribute : to find column that have maximum information gain

par_nodes : to find the next nodes

tree : make tree in return mode, adding nodes to running node and make new nodes and do this every time

predict : calculate the last prediction value of tree

border_set : we set a border number for model to control the number of zero



In [9]:
class tree_node:
    def __init__(self, dataset, target):
        self.dataset = dataset
        self.target = target
        self.par_nodes = []

In [10]:
class DecisionTree:
    def __init__(self, dataset, target, attributes) :
        self.root = tree_node(dataset, target)
        attribute = DecisionTree.Choose_best_Attribute(self.root.dataset, self.root.target, attributes)
        self.root.par_nodes = DecisionTree.par_nodes(self.root, attribute)
        self.root.attribute = attribute
        attributes_left = attributes.copy()
        attributes_left.remove(attribute)
        self.border = 0.5
        for par_node in self.root.par_nodes:
            DecisionTree.tree(par_node, attributes_left)
        return
        
    def Choose_best_Attribute(dataset, target, attributes):
        attribute = attributes[0]
        info_gain = InformationGain(dataset, target, attribute)
        for i in range(1, len(attributes)):
            new_info_gain = InformationGain(dataset, target, attributes[i])
            if new_info_gain > info_gain:
                info_gain = new_info_gain
                attribute = attributes[i]
        return attribute
    
    def par_nodes(node, attribute):
        dataset = node.dataset
        target = node.target
        feature = features[attribute]
        par_nodes = []
        for i in range(feature):
            points = dataset[attribute] == i
            par_nodes.append(tree_node(dataset[points], target[points]))
        return par_nodes
    
    def border_set(self, border):
        self.border = border
        
    def tree(node, attributes):
        if attributes:
            if(len(node.dataset) != 0):
                attribute = DecisionTree.Choose_best_Attribute(node.dataset, node.target, attributes)
                node.par_nodes = DecisionTree.par_nodes(node, attribute)
                node.attribute = attribute
                attributes_left = attributes.copy()
                attributes_left.remove(attribute)
                for par_node in node.par_nodes:
                        DecisionTree.tree(par_node, attributes_left)
                        
    def predict(self, dataset):
        node = self.root
        while (True):
            if len(node.dataset) == 0 or not node.par_nodes:
                break
            attribute = node.attribute
            node = node.par_nodes[dataset[attribute]]
            
        n = len(node.dataset)
        if n == 0:
            return 0
        points = sum(node.target == 1) / n
        if points >= self.border:
            return 1
        else:
            return 0
        
Tree = DecisionTree(tree_train_dataset, tree_train_target, ['last time', 'frequency of donations', 'total blood', 'first time'])

In [11]:
pred_target = []
for i in range(len(tree_test_dataset)):
    prediction = Tree.predict(tree_test_dataset.iloc[i])
    pred_target.append(prediction)
Tree_accuracy, Tree_recall, Tree_F1 = peformance(pred_target, tree_test_target)
print('accuracy = {} \t recall = {} \t F1 = {}'.format(Tree_accuracy, Tree_recall, Tree_F1))

accuracy = 32.666666666666664 	 recall = 32.666666666666664 	 F1 = 49.246231155778894



decrease the border number of model to seperat the 1 targets from zero ones because number of zero in data is very much,after this point whatever we decrease border number we have same result


In [12]:
Tree.border_set(0.05)
pred_target = []
for i in range(len(tree_test_dataset)):
    prediction = Tree.predict(tree_test_dataset.iloc[i])
    pred_target.append(prediction)
Tree_accuracy, Tree_recall, Tree_F1 = peformance(pred_target, tree_test_target)
print('accuracy = {} \t recall = {} \t F1 = {}'.format(Tree_accuracy, Tree_recall, Tree_F1))

accuracy = 46.666666666666664 	 recall = 46.666666666666664 	 F1 = 63.63636363636363



Prepare data for logistic regression

for this we can normlize data, so scaling data between -1 to 1 could normalize that


In [13]:
regression_dataset = pd.DataFrame(data = {'last time': 2 * dataset['last time'] / max(dataset['last time'] - 1),
                                          'frequency of donations' : 2 * dataset['frequency of donations'] / max(dataset['frequency of donations'] - 1),
                                          'total blood' : 2 * dataset['total blood'] / max(dataset['total blood'] - 1),
                                          'first time' : 2 * dataset['first time'] / max(dataset['first time'] - 1)})
regression_dataset = np.c_[regression_dataset, np.ones((len(dataset),1))]
split = int(80 * len(regression_dataset) / 100)
regression_train_dataset, regression_train_target = regression_dataset[:split], np.array(target[:split])
regression_test_dataset, regression_test_target = regression_dataset[split:], np.array(target[split:])
regression_dataset

array([[0.10958904, 0.24489796, 0.2400192 , 0.72164948, 1.        ],
       [0.21917808, 0.6122449 , 0.600048  , 1.58762887, 1.        ],
       [0.08219178, 0.20408163, 0.200016  , 0.53608247, 1.        ],
       ...,
       [0.        , 0.32653061, 0.3200256 , 1.21649485, 1.        ],
       [0.05479452, 0.28571429, 0.2800224 , 1.58762887, 1.        ],
       [0.30136986, 0.32653061, 0.3200256 , 0.84536082, 1.        ]])


logistic regression model

we use gradient descent to rich the answer of logistic because we don't have a closed form answer for model or return mode for that also we set a border number for model to control the number of zero in target like decition tree


In [14]:
class logistic_regression:
    
    def __init__(self, dataset, target):
        self.w = logistic_regression.gradient_descent(dataset, target)
        self.border = 0.5
        
    def border_set(self, border):
        self.border = border
        
    def predict(self, dataset): 
        if 1 / (1 + math.exp(-np.dot(self.w, dataset))) >= self.border: 
            return 1
        else:
            return 0
       
    def gradient_descent(dataset, target):
        w = [0, 0, 0, 0, 0]
        iterations = 100
        learning_rate = 0.01
        for it in range(iterations):
            for i in range(len(dataset)):
                sigmoids = [1 / (1 + math.exp(-np.dot(w, dataset[i])))]
                
            for k in range(len(sigmoids)):
                E = [(sigmoids[k] - target[k]) * dataset[k]]
                
            for i in range(len(sigmoids)):
                w = w - learning_rate * sum([(sigmoids[i] - target[i]) * dataset[i]])
        return w
    
logistic = logistic_regression(regression_train_dataset, regression_train_target)

In [15]:
pred_target = []
for i in range(len(regression_test_dataset)):
    prediction = logistic.predict(regression_test_dataset[i])
    pred_target.append(prediction)
logisitic_accuracy, logisitic_recall, logisitic_F1 = peformance(pred_target, regression_test_target)
print('accuracy = {} \t recall = {} \t F1 = {}'.format(logisitic_accuracy, logisitic_recall, logisitic_F1))

accuracy = 28.000000000000004 	 recall = 28.000000000000004 	 F1 = 43.75000000000001



decrease the border number of model to seperat the 1 targets from zero ones because number of zero in data is very
much,after this point whatever we decrease border number we have same result



In [16]:
logistic.border_set(0.38)
pred_target = []
for i in range(len(regression_test_dataset)):
    prediction = logistic.predict(regression_test_dataset[i])
    pred_target.append(prediction)
logisitic_accuracy, logisitic_recall, logisitic_f1 = peformance(pred_target, regression_test_target)
print('accuracy = {} \t recall = {} \t f1 = {}'.format(logisitic_accuracy, logisitic_recall, logisitic_f1))

accuracy = 48.0 	 recall = 48.0 	 f1 = 64.86486486486487


AdaBoost_Classifier : choose best tree between trees that built on every four features

best_branch : find the best tree with 1 depth and minimum weighted error

training : training a model with m branches


In [17]:
class AdaBoost_Classifier:
    def __init__(self, dataset, target, m):
        self.training(dataset, target, m)
        self.border = 0.3
     
    def border_set(self, border):
        for branch in self.branches:
            branch.border_set(border)
        self.border = border
        
    def training(self, dataset, target, m):
        n = len(dataset)
        w = [1/n] * n
        branches = []
        alphas = []
        for i in range(m):
            branch, branch_error = AdaBoost_Classifier.best_branch(dataset, target, w)
            branches.append(branch)
            beta = branch_error / sum(w)
            alphas.append(math.log((1 - beta) / beta))
            for j in range(len(w)):
                if (branch.predict(dataset.iloc[j]) != target[j]):
                    w[j] = w[j] * math.exp(alphas[i])
        self.branches = branches
        self.alphas = alphas
        
    def predict(self, dataset):
        for i in range(len(self.alphas)):
            prediction = sum([self.alphas[i] * self.branches[i].predict(dataset)])
        if prediction > 0.5:
            return 1
        else: 
            return 0
        
    def best_branch(dataset, target, w):
        attributes = dataset.columns
        tree = DecisionTree(dataset, target, [attributes[0]])
        error = AdaBoost_Classifier.error_base(tree, dataset, target, w)
        for i in range(1, len(attributes)):
            decision_t = DecisionTree(dataset, target, [attributes[i]])
            decision_error = AdaBoost_Classifier.error_base(decision_t, dataset, target, w)
            if decision_error < error:
                error = decision_error
                tree = decision_t
        return tree, error
        
        
    def error_base(tree, dataset, target, w):
        error = 0
        for i in range(len(dataset)):
            prediction = tree.predict(dataset.iloc[i])
            if prediction != target[i]:
                error += w[i]
        return error
                
adaboost = AdaBoost_Classifier(tree_train_dataset, tree_train_target, 4)

In [18]:
pred_target = []
for i in range(len(tree_test_dataset)):
    prediction = adaboost.predict(tree_test_dataset.iloc[i])
    pred_target.append(prediction)
    
adaboost_accuracy, adaboost_recall, adaboost_F1 = peformance(pred_target, tree_test_target)
print('accuracy = {} \t recall = {} \t F1 = {}'.format(adaboost_accuracy, adaboost_recall, adaboost_F1))

accuracy = 28.000000000000004 	 recall = 28.000000000000004 	 F1 = 43.75000000000001


the adaboos model find best tree so we can't change result by changing border number

In [20]:
adaboost.border_set(0.1)
pred_target = []
for i in range(len(tree_test_dataset)):
    prediction = adaboost.predict(tree_test_dataset.iloc[i])
    pred_target.append(prediction)
    
adaboost_accuracy, adaboost_recall, adaboost_F1 = peformance(pred_target, tree_test_target)
print('accuracy = {} \t recall = {} \t F1 = {}'.format(adaboost_accuracy, adaboost_recall, adaboost_F1))

accuracy = 28.000000000000004 	 recall = 28.000000000000004 	 F1 = 43.75000000000001
